In [ ]:
import os
from PIL import Image
import cv2
import tkinter as tk
from tkinter import filedialog
from datetime import datetime
import numpy as np

def select_folder():
    root = tk.Tk()
    root.withdraw()
    parent_folder = filedialog.askdirectory(title="選擇overlay資料夾")
    return parent_folder

def png_to_mp4(folder_path, output_filename, frame_rate):
    # 獲取所有PNG檔案的路徑
    png_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    png_files.sort(key=lambda x: datetime.strptime(x.replace('overlay_', '').split('.')[0], '%Y-%m-%d-%H-%M-%S'))

    # 初始化影片輸出
    height = 1024
    width = 1024
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_filename, fourcc, frame_rate, (width, height))

    # 處理並寫入每張圖像
    for png_file in png_files:
        image_path = os.path.join(folder_path, png_file)
        image = Image.open(image_path)
        image_np = np.array(image)  # 將 PIL 圖像轉換為 NumPy 數組
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)  # 將 RGB 轉換為 BGR

        out.write(image_np)  # 將轉換後的圖像寫入視頻

    # 釋放資源並關閉影片輸出
    out.release()

image_dir = select_folder()
date = image_dir.split('-')[-1]
png_dir = os.path.join(image_dir, 'results')
output_video_path = os.path.join(image_dir, f'RippleDetection_{date}.mp4')
png_to_mp4(png_dir, output_video_path, frame_rate=60)
print(f'RippleDetection_{date}.mp4')


In [ ]:
import cv2
import json
import tkinter as tk
from tkinter import filedialog
import numpy as np
from PIL import Image
import os
from datetime import datetime

def select_folder():
    root = tk.Tk()
    root.withdraw()
    parent_folder = filedialog.askdirectory(title="選擇overlay資料夾")
    return parent_folder

def load_pixel_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return {entry['time']: entry['pixel_number'] for entry in data}

def png_to_mp4(folder_path, output_filename, frame_rate, json_file):
    pixel_data = load_pixel_data(json_file)
    
    # 獲取所有PNG檔案的路徑
    png_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    png_files.sort(key=lambda x: datetime.strptime(x.replace('overlay_', '').split('.')[0], '%Y-%m-%d-%H-%M-%S'))

    # 初始化影片輸出，新增文字區塊
    height = 1024 + 100  # 增加100像素高度用於顯示文字
    width = 1024
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_filename, fourcc, frame_rate, (width, height))

    # 處理並寫入每張圖像
    for png_file in png_files:
        image_path = os.path.join(folder_path, png_file)
        image = Image.open(image_path)
        image_np = np.array(image)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
        
        # 創建帶有文字空間的大圖像
        canvas = np.zeros((height, width, 3), dtype=np.uint8)
        canvas[100:, :, :] = image_np  # 將圖像放在底部
        
        timestamp = png_file.replace('overlay_', '').split('.')[0]
        text = f'Time: {timestamp} - Ripple Counter: {pixel_data[timestamp+".png"]}'
        cv2.putText(canvas, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        out.write(canvas)
    out.release()

image_dir = select_folder()
#image_dir = "C:/Users/user/Desktop/NAS_data/鱸魚/高雄黃明和/frames-20240408"

png_dir = os.path.join(image_dir, 'results')
date = image_dir.split('-')[-1]
output_video_path = os.path.join(image_dir, f'RippleDetection_{date}.mp4')
json_file_path = os.path.join(image_dir, f'pixel_counts_{date}.json')
png_to_mp4(png_dir, output_video_path, frame_rate=60, json_file=json_file_path)


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os

def select_folder():
    root = tk.Tk()
    root.withdraw()
    parent_folder = filedialog.askdirectory(title="選擇overlay資料夾")
    return parent_folder

def merge_videos(directory, output_filename):
    clips = []
    # 掃描目錄並添加所有 MP4 文件到列表
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.mp4'):
            filepath = os.path.join(directory, filename)
            clip = VideoFileClip(filepath)
            clip.reader.fps = 60  # 直接在reader層級設定fps
            clip.fps = 60  # 確保clip物件的fps也被設定
            clips.append(clip)

    # 確保有視頻片段要合併
    if clips:
        final_clip = concatenate_videoclips(clips, method="compose")
        final_clip.fps = 60  # 為最終合併後的視頻設定fps
        final_clip.write_videofile(os.path.join(directory, output_filename), codec='libx264', fps=60)
    else:
        print("No valid video clips found.")


directory = select_folder()
merge_videos(directory, 'Merged_RippleDetection.mp4')


In [ ]:
from PIL import Image, ImageDraw, ImageFont

# 創建一個空白圖像
image = Image.new('RGB', (200, 60), color = (73, 109, 137))
draw = ImageDraw.Draw(image)

# 使用 Noto Color Emoji 字體
font = ImageFont.truetype('C:/Windows/Fonts/BSSYM7.ttf', 32)

# 繪製包含表情符號的文字
draw.text((10, 10), "😀Apple", font=font, fill=(255, 255, 255))

# 顯示圖像
image.show()